In [76]:
from youtube_transcript_api import *
from iso639 import languages
import math

In [126]:
def get_transcript_list(youtube_id):
    try:
        return YouTubeTranscriptApi.list_transcripts(youtube_id)
    except:
        None

def get_langs(transcript_list,manual=True):
    if manual:
        manual_langs = list(transcript_list._manually_created_transcripts.keys())
    else:
        manual_langs = list(transcript_list._generated_transcripts.keys())
        
    langs = []
    for lang_code in manual_langs:
        try:
            langs.append(languages.get(alpha2=lang_code).name)
        except:
            pass
    return langs

def get_transcript(transcript_list,lang):
    return transcript_list.find_transcript([lang]).fetch()

def get_time_stamps(transcript,word):
    time_stamps = []
    for sentence in transcript:
        if word in sentence["text"].lower():
            time = sentence["start"]
            duration = int(sentence["duration"])
            time_formated = ''
            if time < 60:
                secs = int(time)
                time_formated = f'00:00:{secs} - 00:00:{secs+duration}'
            elif 60 <= time < 3600:
                mins = int(time/60)
                secs = int((time % 60) * 60)
                secs_lenght = len(str(secs))
                secs = secs if secs_lenght < 2 else int(secs / pow(10,secs_lenght-2))
                time_formated = f'00:{mins}:{secs} - 00:{mins}:{secs+duration}'
            else:
                hours = int(time/3600)
                mins = int((time % 3600) * 60)
                secs = int((time % 3600) * 3600) % 100
                secs_lenght = len(str(secs))
                secs = secs if secs_lenght < 2 else int(secs / pow(10,secs_lenght-2))
                time_formated = f'{hours}:{mins}:{secs} - {hours}:{mins}:{secs+duration}'
            time_stamps.append(time_formated)

    return time_stamps

def get_translation_langs(transcript_list):
    translation_langs_code = []
    for lang in transcript_list._translation_languages:
        try:
            translation_langs_code.append(lang["language_code"])
        except:
            pass
    return translation_langs_code